In [1]:
import os
import pandas as pd
from dbfread import DBF

In [7]:
dfAnalysisSets = pd.read_csv('output\\_analysis_sets.csv')
dfAnalysisSets = dfAnalysisSets[dfAnalysisSets['zone_set_name'].isin(['Summit Wasatch Rec'])]
dfAnalysisSets

,date_range_name,data_ranges,mode_name,mode_description,zone_set_name,analysis_type,analysis_set_name
1204,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",TRU,Truck,Summit Wasatch Rec,Pre-Set OD,Recreation_Zones_Summit-Wasatch_2019_01_TRU
1205,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",BIC,Bicycle,Summit Wasatch Rec,Pre-Set OD,Recreation_Zones_Summit-Wasatch_2019_01_BIC
1206,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",PED,Pedestrian,Summit Wasatch Rec,Pre-Set OD,Recreation_Zones_Summit-Wasatch_2019_01_PED
1207,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",ALL,All_Vehicles,Summit Wasatch Rec,Pre-Set OD,Recreation_Zones_Summit-Wasatch_2019_01_ALL
1208,2019_02,"[{'start_date': '02/01/2019', 'end_date': '02/...",TRU,Truck,Summit Wasatch Rec,Pre-Set OD,Recreation_Zones_Summit-Wasatch_2019_02_TRU
...,...,...,...,...,...,...,...
1371,2022_03,"[{'start_date': '03/01/2022', 'end_date': '03/...",ALL,All_Vehicles,Summit Wasatch Rec,Pre-Set OD,Recreation_Zones_Summit-Wasatch_2022_03_ALL
1372,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",TRU,Truck,Summit Wasatch Rec,Pre-Set OD,Recreation_Zones_Summit-Wasatch_2022_04_TRU
1373,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",BIC,Bicycle,Summit Wasatch Rec,Pre-Set OD,Recreation_Zones_Summit-Wasatch_2022_04_BIC
1374,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",PED,Pedestrian,Summit Wasatch Rec,Pre-Set OD,Recreation_Zones_Summit-Wasatch_2022_04_PED


In [8]:
# read DBFs to get zones
zoneSetDir = os.path.join(os.getcwd(), r'Zone-Sets\redownloaded-from-streetlight')
analysisSetDir = os.path.join(os.getcwd(), r'output\results-by-analysis')
indivZonesDataDir = os.path.join(os.getcwd(), r'output\results-by-indiv-zones')

dfZoneSets = dfAnalysisSets[['zone_set_name']].drop_duplicates()

dfZoneSets

for indexZS, rowZS in dfZoneSets.iterrows():
    zsName = rowZS['zone_set_name'].replace(' ','-')
    dbfName = (zoneSetDir + "\\UDOT---PPU_" + zsName + '_ZoneSet\\zone_set_' + zsName.replace('-','_') + '.dbf').replace('_Moab.dbf','.dbf')
    print(zsName)

    _dfZones = pd.DataFrame(DBF(dbfName, load=True))
    _dfZones['name'] = _dfZones['name'].astype(str)

    #display(_dfZoneSet)
    dfAnalysisSets_ZoneSet = dfAnalysisSets[dfAnalysisSets['zone_set_name']==rowZS['zone_set_name']]

    _dfZoneSetResults = pd.DataFrame()

    # read in data for each zone set
    for indexA, rowA in dfAnalysisSets_ZoneSet.iterrows():
        #print(rowA['analysis_set_name'])

        _df = pd.read_csv(os.path.join(analysisSetDir,rowA['analysis_set_name']+'.csv'),low_memory=False)

        _df['mode_name'] = rowA['mode_name']
        _df['date_range_name'] = rowA['date_range_name']

        if(rowA['mode_name']!='TRU'):
            _df['truck_weight'] = 'na'

        _dfZoneSetResults = pd.concat([_dfZoneSetResults,_df])

    col = _dfZoneSetResults.pop('truck_weight')  # remove column 'B' from the dataframe and store it in 'col'
    _dfZoneSetResults.insert(len(_dfZoneSetResults.columns), 'truck_weight', col)  # insert column 'B' at the end of the dataframe
    
    _dfZoneSetResults['OName'] = _dfZoneSetResults['OName'].astype(str)
    _dfZoneSetResults['DName'] = _dfZoneSetResults['DName'].astype(str)
    #display(_dfZoneSetResults)

    # write out files for each zone
    for indexZ, rowZ in _dfZones.iterrows():
        print(rowZ['name'])

        _dfZoneDataAsO = _dfZoneSetResults[_dfZoneSetResults['OName']==rowZ['name']]
        _dfZoneDataAsD = _dfZoneSetResults[_dfZoneSetResults['DName']==rowZ['name']]

        _dfZoneDataAsO.to_csv(indivZonesDataDir + rowZS['zone_set_name'] + '__' + rowZ['name'] + '_asOrig.csv',index=False)
        _dfZoneDataAsD.to_csv(indivZonesDataDir + rowZS['zone_set_name'] + '__' + rowZ['name'] + '_asDest.csv',index=False)

Summit-Wasatch-Rec
PCMR_Base
Canyons_Park2
Echo_Park1
Canyons_Park3
UOP_Park1
DC_Park1
DC_Park2
DC_Park3
DV_Park1
Jordanelle_Park1
Rockport_Park1
Strawberry_Park1
Echo_Park2
Strawberry_Park2
Strawberry_Park3
Wasatch_Park1
Wasatch_Park2
Wasatch_Park3
SoldierH_1
